In [ ]:
import firedrake
import folium
import matplotlib.pyplot as plt
import numpy as np
import pyproj
import ufl

In [ ]:
import femlium

Auxiliary function to get a `folium` `Map` close to Lake Garda.

In [ ]:
def get_garda_geo_map(boundary_icons: bool = False) -> folium.Map:
    """Get a map close to Lake Garda, and possibly add some boundary markers."""
    # Add map close to Lake Garda
    geo_map = folium.Map(location=[45.6389113, 10.7521368], zoom_start=10.3)

    # Add markers
    if boundary_icons:
        location_markers = {
            "Sarca": [45.87395405, 10.87087005],
            "Mincio": [45.43259035, 10.7007715]
        }
        location_colors = {
            "Sarca": "red",
            "Mincio": "green"
        }

        for key in location_markers.keys():
            folium.Marker(
                location=location_markers[key],
                tooltip=key,
                icon=folium.Icon(color=location_colors[key])
            ).add_to(geo_map)

    # Return folium map
    return geo_map

In [ ]:
get_garda_geo_map()

Read the mesh from file with `firedrake`.

In [ ]:
mesh = firedrake.Mesh("data/garda.msh")

Plot the mesh using `firedrake.triplot`.

In [ ]:
fig = plt.figure(figsize=(12, 12))
firedrake.triplot(mesh, axes=fig.gca())
fig.gca().axis("equal")

Define a `pyproj` `Transformer` to map between different reference systems, because the points read from file are stored a $(x, y)$ pairs in the EPSG32632 reference system, while the map produced by `folium` is based on (latitude, longitude) pairs in the EPSG4326 reference system.

In [ ]:
transformer = pyproj.Transformer.from_crs("epsg:32632", "epsg:4326", always_xy=True)

We define a mesh plotter for meshes in `firedrake` format, which is implemented in `femlium.FiredrakePlotter`.

In [ ]:
firedrake_plotter = femlium.FiredrakePlotter(transformer)

We use the `firedrake_plotter` to draw the mesh on top of the geographic map.

In [ ]:
geo_map = get_garda_geo_map()
firedrake_plotter.add_mesh_to(geo_map, mesh)
geo_map

We may change the color and the weight of the line.

In [ ]:
geo_map = get_garda_geo_map()
firedrake_plotter.add_mesh_to(geo_map, mesh, face_colors="red", face_weights=2)
geo_map

Furthermore, we may set the colors and the weights of the face representation to depend on the markers associated to each segment.

In [ ]:
geo_map = get_garda_geo_map(boundary_icons=True)
face_colors = {
    0: "gray",
    1: "blue",
    2: "red",
    3: "green"
}
face_weights = {
    0: 1,
    1: 2,
    2: 5,
    3: 5
}
firedrake_plotter.add_mesh_to(geo_map, mesh, face_colors=face_colors, face_weights=face_weights)
geo_map

Cells can be colored as well, with a uniform color or depending on the cell markers. We start from a uniform color.

In [ ]:
geo_map = get_garda_geo_map()
firedrake_plotter.add_mesh_to(geo_map, mesh, cell_colors="orange")
geo_map

We also show the case of colors being set from cell markers. There are two cell markers in this mesh, equal to 1 for the region close to the shoreline (colored in purple) and 2 for the rest of the domain (colored in yellow).

In [ ]:
geo_map = get_garda_geo_map()
cell_colors = {
    1: "purple",
    2: "yellow"
}
firedrake_plotter.add_mesh_to(geo_map, mesh, cell_colors=cell_colors)
geo_map

Once can use colors associated to both cell and face markers on the same plot. 

In [ ]:
geo_map = get_garda_geo_map(boundary_icons=True)
firedrake_plotter.add_mesh_to(
    geo_map, mesh,
    cell_colors=cell_colors, face_colors=face_colors, face_weights=face_weights)
geo_map

In order to define a simple scalar field, we compute the centroid of the domain.

In [ ]:
centroid = np.mean(mesh.coordinates.dat.data_ro, axis=0)

We may plot the centroid on top of the mesh.

In [ ]:
geo_map = get_garda_geo_map()
firedrake_plotter.add_mesh_to(geo_map, mesh)
folium.Marker(location=transformer.transform(*centroid)[::-1], tooltip="Centroid").add_to(geo_map)
geo_map

The scalar field is defined as $s(\rho, \theta) = \frac{\rho}{\sqrt{1 - 0.5 \cos^2 \theta}}$, and is interpolated on a $\mathbb{P}^2$ finite element space. Here $(\rho, \theta)$ are the polar coordinates centered at the centroid.

In [ ]:
scalar_function_space = firedrake.FunctionSpace(mesh, "CG", 2)

In [ ]:
x = ufl.SpatialCoordinate(mesh)
rho = ufl.sqrt((x[0] - centroid[0])**2 + (x[1] - centroid[1])**2)
theta = ufl.atan_2(x[1] - centroid[1], x[0] - centroid[0])
scalar_field = firedrake.interpolate(rho / ufl.sqrt(1 - 0.5 * ufl.cos(theta)**2), scalar_function_space)

We next show a filled contour plot with 15 levels using `firedrake.tricontourf`.

In [ ]:
fig = plt.figure(figsize=(12, 12))
trif = firedrake.tricontourf(scalar_field, axes=fig.gca(), levels=15, cmap="jet")
fig.colorbar(trif)
fig.gca().axis("equal")

In order to plot a field on a geographic map, we use again the `firedrake_plotter`. We may plot the same filled contour plot on the geographic map.

In [ ]:
geo_map = get_garda_geo_map()
firedrake_plotter.add_scalar_field_to(geo_map, scalar_field, mode="contourf", levels=15, cmap="jet")
geo_map

Similarly, we can also use (unfilled) contour plots.

In [ ]:
fig = plt.figure(figsize=(12, 12))
tri = firedrake.tricontour(scalar_field, axes=fig.gca(), levels=15, cmap="jet")
fig.colorbar(tri)
fig.gca().axis("equal")

In [ ]:
geo_map = get_garda_geo_map()
firedrake_plotter.add_scalar_field_to(geo_map, scalar_field, mode="contour", levels=15, cmap="jet")
geo_map

One may also combine mesh plots and solution plots.

In [ ]:
geo_map = get_garda_geo_map()
firedrake_plotter.add_mesh_to(geo_map, mesh, face_colors="grey")
firedrake_plotter.add_scalar_field_to(geo_map, scalar_field, mode="contour", levels=15, cmap="jet")
geo_map

We next define a vector field $\mathbf{v}(\rho, \theta) = \begin{bmatrix}-\rho \sin \theta\\\rho \cos\theta \end{bmatrix}$.

In [ ]:
vector_function_space = firedrake.VectorFunctionSpace(mesh, "CG", 2)

In [ ]:
vector_field = firedrake.interpolate(
    ufl.as_vector((- rho * ufl.sin(theta), rho * ufl.cos(theta))), vector_function_space)

We may obtain contourf or contour plots of the magnitude of the vector field.

In [ ]:
geo_map = get_garda_geo_map()
firedrake_plotter.add_vector_field_to(geo_map, vector_field, mode="contourf", levels=15, cmap="jet")
geo_map

In [ ]:
geo_map = get_garda_geo_map()
firedrake_plotter.add_vector_field_to(geo_map, vector_field, mode="contour", levels=15, cmap="jet")
geo_map

Vector field can also be plotted using a quiver. We first see the quiver plot obtained with `firedrake.quiver`.

In [ ]:
fig = plt.figure(figsize=(12, 12))
quiv = firedrake.quiver(vector_field, axes=fig.gca(), cmap="jet")
fig.colorbar(quiv)
fig.gca().axis("equal")

A similar plot can rendered on top of the geographic map.

In [ ]:
geo_map = get_garda_geo_map()
firedrake_plotter.add_vector_field_to(geo_map, vector_field, mode="quiver", scale=1e-1, cmap="jet")
geo_map